In [1]:
import pandas as pd

data=pd.read_csv('dataset.csv',sep=',')
data=data.drop('date',axis=1)

In [2]:
data

,dow_jones_open_price,dow_jones_high_price,dow_jones_low_price,dow_jones_close_price,dow_jones_adj_close_price,dow_jones_trading_volume,nasdaq_open_price,nasdaq_high_price,nasdaq_low_price,nasdaq_close_price,nasdaq_adj_close_price,nasdaq_trading_volume,sp_open_price,sp_high_price,sp_low_price,sp_close_price,sp_adj_close_price,sp_trading_volume,usd_to_eur_exchange_rate,usd_to_uk_exchange_rate,jp_to_usd_exchange_rate,brent_crude_oil_price,cushing_crude_oil_price,treasury_1_year_rate,treasury_10_year_rate,breakeven_inflation_5_year,breakeven_inflation_10_year,bank_prime_loan_rate,federal_funds_rate,natural_gas_futures_contract_1_price,natural_gas_futures_contract_2_price,natural_gas_futures_contract_3_price,natural_gas_futures_contract_4_price,cushing_crude_futures_contract_1_price,cushing_crude_futures_contract_2_price,cushing_crude_futures_contract_3_price,cushing_crude_futures_contract_4_price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8342.38,8608.27,8342.38,8607.52,8607.52,243910000.0,1346.93,1384.91,1336.98,1384.85,1384.85,1.287540e+09,879.82,909.03,879.82,909.03,909.03,1.229200e+09,1.0361,1.5975,119.86,30.32,31.97,1.42,4.07,1.30,1.64,4.25,1.30,5.251,5.131,4.821,4.691,31.85,31.10,29.91,28.84
2,8607.38,8635.49,8552.87,8601.69,8601.69,234000000.0,1382.36,1389.44,1374.61,1387.08,1387.08,1.149590e+09,909.03,911.25,903.07,908.59,908.59,1.130800e+09,1.0418,1.6045,119.85,31.43,33.26,1.41,4.05,1.28,1.62,4.25,1.12,5.344,5.220,4.945,4.795,33.08,32.22,30.84,29.62
3,8602.78,8800.59,8602.10,8773.57,8773.57,258040000.0,1390.19,1428.65,1390.09,1421.32,1421.32,1.567690e+09,908.59,931.77,908.59,929.01,929.01,1.435900e+09,1.0469,1.6080,119.06,31.43,32.29,1.44,4.09,1.31,1.63,4.25,1.22,4.935,4.870,4.731,4.633,32.10,31.44,30.17,28.99
4,8775.84,8802.64,8713.03,8740.59,8740.59,266970000.0,1424.26,1442.26,1416.23,1431.57,1431.57,1.755760e+09,929.01,930.81,919.93,922.93,922.93,1.545200e+09,1.0419,1.6055,120.18,30.78,31.20,1.40,4.04,1.28,1.62,4.25,1.20,5.127,5.047,4.875,4.755,31.08,30.55,29.63,28.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5814,40527.82,40791.18,40346.94,40368.96,40368.96,529800000.0,16842.39,16979.96,16753.22,16823.17,16823.17,7.483050e+09,5411.99,5450.41,5386.44,5396.63,5396.63,4.317110e+09,1.1290,1.3223,143.10,NaN,NaN,3.99,4.35,2.31,2.19,7.50,4.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5815,40179.49,40416.80,39394.75,39669.39,39669.39,737550000.0,16499.69,16600.37,16066.46,16307.16,16307.16,7.909190e+09,5335.75,5367.24,5220.79,5275.70,5275.70,4.607750e+09,1.1382,1.3247,142.56,NaN,NaN,3.96,4.29,2.29,2.17,7.50,4.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5816,39745.58,39745.58,38950.31,39142.23,39142.23,657400000.0,16399.97,16408.51,16181.17,16286.45,16286.45,7.046290e+09,5305.45,5328.31,5255.58,5282.70,5282.70,4.714880e+09,1.1364,1.3259,142.33,NaN,NaN,3.99,4.34,2.26,2.23,7.50,4.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1390,1.3279,142.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# List unique values for each attribute
unknown_in_attribute=[0 for i in range(5819)]
for column in data.columns:
    unique_vals = data[column].isna()
    i=0
    for boolean in unique_vals:
        if boolean:
            unknown_in_attribute[i]+=1
        i+=1

In [4]:
set_try={}
for num in set(unknown_in_attribute):
    set_try[num]=0
for num in unknown_in_attribute:
    set_try[num]+=1

In [5]:
data.insert(loc=0,column='num_unknown',value=unknown_in_attribute)

In [6]:
new_data=data[data['num_unknown']==0].copy()

In [7]:
new_data=new_data.drop(columns='num_unknown',axis=1)

In [8]:
target=new_data[' cushing_crude_oil_price'].copy()
X=new_data.drop(columns=[' cushing_crude_oil_price',' brent_crude_oil_price'],axis=1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error

gnb=SVR()
X_train, X_test, y_train, y_test=train_test_split(X,target,test_size=0.3,random_state=43)

gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)

print(root_mean_squared_error(y_test,y_pred))

21.277930282134555


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5236 entries, 1 to 5547
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0    dow_jones_open_price                    5236 non-null   float64
 1    dow_jones_high_price                    5236 non-null   float64
 2    dow_jones_low_price                     5236 non-null   float64
 3    dow_jones_close_price                   5236 non-null   float64
 4    dow_jones_adj_close_price               5236 non-null   float64
 5    dow_jones_trading_volume                5236 non-null   float64
 6    nasdaq_open_price                       5236 non-null   float64
 7    nasdaq_high_price                       5236 non-null   float64
 8    nasdaq_low_price                        5236 non-null   float64
 9    nasdaq_close_price                      5236 non-null   float64
 10   nasdaq_adj_close_price                  5236 non-nul

In [11]:
provider=['dow_jones','nasdaq','sp']
tag=['open','high','low','close','adj_close']
another=['trading_volume']
exchange_rate=['usd_to_eur','usd_to_uk','jp_to_usd']
treasure_year=[1,10]
inflation=[5,10]
rate=['bank_prime_loan','federal_funds']
separated=['dow_jones','nasdaq','sp','exchange_rate','treasure_year','inflation','natural_gas_price','cushing_crude_price','another_rate']

fill=[]
for pro in provider:
    que=[]
    for t in tag:
        que.append(' '+pro+"_"+t+"_price")
    for a in another:
        que.append(' '+pro+"_"+a)
    fill.append(que)
shift=[]
for ex in exchange_rate:
    shift.append(' '+ex+"_exchange_rate")
fill.append(shift)
shift=[]
for i in treasure_year:
    shift.append(f" treasury_{i}_year_rate")
fill.append(shift)
shift=[]
for i in inflation:
    shift.append(f" breakeven_inflation_{i}_year")
fill.append(shift)
for name in ['natural_gas','cushing_crude']:
    shift=[]
    for i in range(1,5):
        shift.append(f" {name}_futures_contract_{i}_price")
    fill.append(shift)
shift=[]
for r in rate:
    shift.append(" "+r+"_rate")
fill.append(shift)
fill

[[' dow_jones_open_price',
  ' dow_jones_high_price',
  ' dow_jones_low_price',
  ' dow_jones_close_price',
  ' dow_jones_adj_close_price',
  ' dow_jones_trading_volume'],
 [' nasdaq_open_price',
  ' nasdaq_high_price',
  ' nasdaq_low_price',
  ' nasdaq_close_price',
  ' nasdaq_adj_close_price',
  ' nasdaq_trading_volume'],
 [' sp_open_price',
  ' sp_high_price',
  ' sp_low_price',
  ' sp_close_price',
  ' sp_adj_close_price',
  ' sp_trading_volume'],
 [' usd_to_eur_exchange_rate',
  ' usd_to_uk_exchange_rate',
  ' jp_to_usd_exchange_rate'],
 [' treasury_1_year_rate', ' treasury_10_year_rate'],
 [' breakeven_inflation_5_year', ' breakeven_inflation_10_year'],
 [' natural_gas_futures_contract_1_price',
  ' natural_gas_futures_contract_2_price',
  ' natural_gas_futures_contract_3_price',
  ' natural_gas_futures_contract_4_price'],
 [' cushing_crude_futures_contract_1_price',
  ' cushing_crude_futures_contract_2_price',
  ' cushing_crude_futures_contract_3_price',
  ' cushing_crude_future

In [12]:
dow_jones_data=X[fill[0]]
nasdaq_data=X[fill[1]]
sp_data=X[fill[2]]

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
used_model={
    "LinearRegression": LinearRegression(),
    "SVR":SVR(),
    "Decision Tree REgression": DecisionTreeRegressor(),
    "Gaussian Processs": GaussianProcessRegressor()
}

In [15]:

for name in used_model.keys():
    print(name+"............................")
    i=0
    for sep in separated:
        X_train, X_test, y_train, y_test=train_test_split(X[fill[i]],target,test_size=0.3,random_state=43)

        model=used_model[name]
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)

        print(f"With {sep} data, we have rmse: {root_mean_squared_error(y_test,y_pred)}")
        i+=1
    print('............................................')


LinearRegression............................
With dow_jones data, we have rmse: 22.72026526541641
With nasdaq data, we have rmse: 23.352306685736178
With sp data, we have rmse: 23.021914267877307
With exchange_rate data, we have rmse: 18.252636500020937
With treasure_year data, we have rmse: 23.360155033771218
With inflation data, we have rmse: 18.968099995148823
With natural_gas_price data, we have rmse: 22.49685222799659
With cushing_crude_price data, we have rmse: 0.547616406883211
With another_rate data, we have rmse: 23.20707585856249
............................................
SVR............................
With dow_jones data, we have rmse: 23.90663468106503
With nasdaq data, we have rmse: 23.83034787101832
With sp data, we have rmse: 21.461642695597696
With exchange_rate data, we have rmse: 21.29695194530938
With treasure_year data, we have rmse: 20.357339724373244
With inflation data, we have rmse: 17.898199820832172
With natural_gas_price data, we have rmse: 20.634270889606

In [18]:
new_data

,dow_jones_open_price,dow_jones_high_price,dow_jones_low_price,dow_jones_close_price,dow_jones_adj_close_price,dow_jones_trading_volume,nasdaq_open_price,nasdaq_high_price,nasdaq_low_price,nasdaq_close_price,nasdaq_adj_close_price,nasdaq_trading_volume,sp_open_price,sp_high_price,sp_low_price,sp_close_price,sp_adj_close_price,sp_trading_volume,usd_to_eur_exchange_rate,usd_to_uk_exchange_rate,jp_to_usd_exchange_rate,brent_crude_oil_price,cushing_crude_oil_price,treasury_1_year_rate,treasury_10_year_rate,breakeven_inflation_5_year,breakeven_inflation_10_year,bank_prime_loan_rate,federal_funds_rate,natural_gas_futures_contract_1_price,natural_gas_futures_contract_2_price,natural_gas_futures_contract_3_price,natural_gas_futures_contract_4_price,cushing_crude_futures_contract_1_price,cushing_crude_futures_contract_2_price,cushing_crude_futures_contract_3_price,cushing_crude_futures_contract_4_price
1,8342.38,8608.27,8342.38,8607.52,8607.52,243910000.0,1346.93,1384.91,1336.98,1384.85,1384.85,1.287540e+09,879.82,909.03,879.82,909.03,909.03,1.229200e+09,1.0361,1.5975,119.86,30.32,31.97,1.42,4.07,1.30,1.64,4.25,1.30,5.251,5.131,4.821,4.691,31.85,31.10,29.91,28.84
2,8607.38,8635.49,8552.87,8601.69,8601.69,234000000.0,1382.36,1389.44,1374.61,1387.08,1387.08,1.149590e+09,909.03,911.25,903.07,908.59,908.59,1.130800e+09,1.0418,1.6045,119.85,31.43,33.26,1.41,4.05,1.28,1.62,4.25,1.12,5.344,5.220,4.945,4.795,33.08,32.22,30.84,29.62
3,8602.78,8800.59,8602.10,8773.57,8773.57,258040000.0,1390.19,1428.65,1390.09,1421.32,1421.32,1.567690e+09,908.59,931.77,908.59,929.01,929.01,1.435900e+09,1.0469,1.6080,119.06,31.43,32.29,1.44,4.09,1.31,1.63,4.25,1.22,4.935,4.870,4.731,4.633,32.10,31.44,30.17,28.99
4,8775.84,8802.64,8713.03,8740.59,8740.59,266970000.0,1424.26,1442.26,1416.23,1431.57,1431.57,1.755760e+09,929.01,930.81,919.93,922.93,922.93,1.545200e+09,1.0419,1.6055,120.18,30.78,31.20,1.40,4.04,1.28,1.62,4.25,1.20,5.127,5.047,4.875,4.755,31.08,30.55,29.63,28.73
5,8735.93,8736.07,8580.10,8595.31,8595.31,255650000.0,1423.29,1424.12,1399.06,1401.07,1401.07,1.457010e+09,922.93,922.93,908.32,909.93,909.93,1.467600e+09,1.0447,1.6063,119.35,29.30,30.66,1.36,4.00,1.33,1.71,4.25,1.29,5.161,5.111,4.933,4.813,30.56,30.06,29.29,28.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,39763.74,39868.59,39717.25,39807.37,39807.37,360970000.0,16377.23,16420.92,16347.38,16379.46,16379.46,5.174450e+09,5248.03,5264.85,5245.82,5254.35,5254.35,3.998270e+09,1.0801,1.2634,151.35,86.17,83.96,5.03,4.20,2.38,2.32,8.50,5.33,1.763,1.997,2.340,2.442,83.17,82.42,81.67,80.89
5544,39256.27,39256.27,39051.70,39170.24,39170.24,319590000.0,16199.24,16251.41,16137.24,16240.45,16240.45,4.710280e+09,5204.29,5208.34,5184.05,5205.81,5205.81,3.886590e+09,1.0769,1.2576,151.55,87.63,85.95,5.05,4.36,2.44,2.36,8.50,5.33,1.862,2.105,2.430,2.540,85.15,84.22,83.29,82.34
5545,39139.59,39305.76,39017.13,39127.14,39127.14,364180000.0,16178.80,16340.74,16169.31,16277.46,16277.46,5.151500e+09,5194.37,5228.75,5194.37,5211.49,5211.49,3.703250e+09,1.0827,1.2635,151.67,90.60,86.22,5.03,4.36,2.43,2.36,8.50,5.33,1.841,2.067,2.386,2.494,85.43,84.61,83.77,82.88
5546,39343.60,39421.35,38559.42,38596.98,38596.98,356510000.0,16418.57,16468.04,16046.56,16049.08,16049.08,5.368700e+09,5244.05,5256.59,5146.06,5147.21,5147.21,4.075680e+09,1.0865,1.2669,151.66,90.32,87.37,5.00,4.31,2.42,2.34,8.50,5.33,1.774,1.997,2.324,2.430,86.59,85.81,84.96,84.03
